# Collision Avoidance - Live Demo
In this notebook we'll use the model we trained to detect whether the robot is free or blocked to enable a collision avoidance behavior on the robot. It is assumed that the optimized neural network has already been created and loaded into the Jupyter Lab file browser.

## TensorRT

In [1]:
import cv2
import torch
from torchvision.models import resnet18
from torch2trt import TRTModule
from jetracer.nvidia_racecar import NvidiaRacecar
from jetcam.csi_camera import CSICamera
# from utils import preprocess
import torch.nn.functional as F
import numpy as np
import time

Load the optimized model by executing the cell below.

In [2]:
model_trt = TRTModule()
model_trt.load_state_dict(torch.load('best_model_trt.pth'))

<All keys matched successfully>

Inizialize `car` and `camera` objects.

In [3]:
car = NvidiaRacecar()
camera = CSICamera(width=224, height=224, capture_fps=65)

In [8]:
import torch
import cv2
from torchvision import transforms

# Chọn device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 3)  # 3 lớp

checkpoint = torch.load('best_model_resnet18_3class.pth', map_location=device)
model.load_state_dict(checkpoint)   # hoặc checkpoint['model_state'] nếu bạn lưu dạng dict
model = model.to(device).eval()

# Transform giống khi train model
transform_pipeline = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

def preprocess(image):
    # Resize về 224x224 nếu cần
    image_resized = cv2.resize(image, (224, 224))
    # Đưa về tensor và thêm batch dimension
    return transform_pipeline(image_resized).unsqueeze(0).to(device)


NameError: name 'models' is not defined

In [7]:
import torch.nn.functional as F

THROTTLE_NORMAL = 0.2
THROTTLE_BOOST = 0.35

while True:
    image = camera.value  # nếu dùng CSICamera, hoặc camera.read() nếu USB
    if image is None:
        continue

    image_proc = preprocess(image).half()

    with torch.no_grad():
        output_obstacle = model_obstacle(image_proc)
        probs = torch.softmax(output_obstacle, dim=1)
        idx = torch.argmax(probs, dim=1).item()

    print(f"Class: {classes[idx]} - Conf: {probs[0, idx].item()*100:.1f}%")



NameError: name 'model_obstacle' is not defined

In [ ]:
import torch.nn.functional as F

# Các tham số
STEERING_GAIN = 0.9
STEERING_BIAS = 0
THROTTLE_NORMAL = 0.2
THROTTLE_BOOST = 0.35

car.throttle = 0.0
car.steering = 0.0

while True:
    image = camera.read()
    image_proc = preprocess(image).half()

    # Dự đoán vật cản (3 lớp: blocked, normal, boost)
    output_obstacle = model_obstacle(image_proc)
    probs = F.softmax(output_obstacle, dim=1).detach().cpu().numpy().flatten()
    predicted_class = int(np.argmax(probs))

    # Dự đoán điều khiển lái
    output_lane = model_steering(image_proc).detach().cpu().numpy().flatten()
    steering = float(output_lane[0])
    car.steering = steering * STEERING_GAIN + STEERING_BIAS

    # Điều chỉnh tốc độ theo nhãn
    if predicted_class == 0:  # blocked
        car.throttle = 0.0
    elif predicted_class == 1:  # normal
        car.throttle = THROTTLE_NORMAL
    elif predicted_class == 2:  # boost
        car.throttle = THROTTLE_BOOST

    time.sleep(0.01)


## Live Demo

Due to the JetRacer's limited steering angle and obstacle visibility during the steering process, a specific time interval is required for successful obstacle avoidance. In the suggested demo, the robot will steer for 2 seconds upon obstacle detection. Nevertheless, this value should be dynamically modified to align with specific operational needs.

In [ ]:
prob_blocked = 0
threshold = 0.5

turning = False  # Flag to indicate if the car is turning
turn_time = time.time()  # Start a timer

while True:
    image = camera.read()
#     image_proc = preprocess(image).half()
    output = model_trt(image)
    y = F.softmax(output, dim=1)
    prob_blocked = float(y.flatten()[0])

    if prob_blocked < threshold :
        car.steering = 0.0
    else:
        if not turning:
            car.steering = -0.8  # Choose a steering angle
            turning = True  # Set the turning flag
            turn_time = time.time()  # Start a timer

        if time.time() - turn_time >= 2 and prob_blocked < threshold:
            car.steering = 0.0  # Straighten the wheels
            turning = False  # Clear the turning flag

    car.throttle = 0.18  # Choose car.throttle